In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from sklearn.cluster import KMeans
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.metrics import silhouette_score
from scipy.sparse import lil_matrix, csr_matrix
import numpy as np
import matplotlib.pyplot as plt
import re
import numpy as np
import pandas as pd
from collections import Counter, defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.model_selection import KFold


import scipy
from scipy.sparse import csr_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
%pip install seaborn
import seaborn as sns

# For nice plots in Jupyter:
%matplotlib inline

# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')


In [ ]:
# Replace 'text8.txt' with the path to your file.
filename = "text8.txt"

# Read file
with open(filename, "r", encoding="utf-8") as f:
    text = f.read()

# Basic cleaning: lowercase, remove punctuation, digits, and non-ASCII chars
def clean_text(txt):
    txt = txt.lower()
    txt = re.sub(r'[^\x00-\x7F]+', ' ', txt)  # remove non-ASCII
    txt = re.sub(r'\d+', '', txt)            # remove digits
    txt = re.sub(r'[^\w\s]', '', txt)        # remove punctuation
    return txt

text = clean_text(text)

# Tokenize
tokens = word_tokenize(text)

# (Optional) Remove stopwords
stop_words = set(stopwords.words('english'))
tokens = [t for t in tokens if t not in stop_words and t.strip() != ""]

print(f"Number of tokens after cleaning and stopword removal: {len(tokens)}")
print("Sample tokens:", tokens[:30])

In [ ]:
# Count frequency of all tokens
freq = Counter(tokens)

# Sort by frequency (descending)
most_common = freq.most_common()

# Example: choose top 5000 words (adjust as needed to control memory usage)
vocab_size = 5000
vocab_list = [w for w, _ in most_common[:vocab_size]]

# Create a mapping (word -> index)
vocab = {word: idx for idx, word in enumerate(vocab_list)}

print("Vocab size:", len(vocab))
print("First 20 vocab words:", vocab_list[:20])

In [ ]:
from sklearn.model_selection import train_test_split

# Filter tokens to include only the top 5000 most frequent words
filtered_tokens = [t for t in tokens if t in vocab_list]

# Split the filtered tokens into training and validation sets
train_tokens, val_tokens = train_test_split(filtered_tokens, test_size=0.2, random_state=42)

# Verify the split
print(f"Number of training tokens: {len(train_tokens)}")
print(f"Number of validation tokens: {len(val_tokens)}")

In [ ]:
# Proceed with building the co-occurrence matrix on the training data
window_size = 2
train_vocab = sorted(set(train_tokens))
word_index = {word: idx for idx, word in enumerate(train_vocab)}

# Proceed with the co-occurrence matrix using train_tokens
co_matrix_sparse = lil_matrix((len(train_vocab), len(train_vocab)), dtype=int)

for i, word in enumerate(train_tokens):
    for j in range(max(0, i - window_size), min(len(train_tokens), i + window_size + 1)):
        if i != j:
            co_matrix_sparse[word_index[word], word_index[train_tokens[j]]] += 1

# Convert to compressed sparse row format
co_matrix_sparse = csr_matrix(co_matrix_sparse)

print(f"Vocabulary size: {len(train_vocab)}")
print(f"Non-zero entries in the co-occurrence matrix: {co_matrix_sparse.nnz}")

In [ ]:
# Normalize the sparse matrix
normalized_matrix = normalize(co_matrix_sparse, norm='l2', axis=1)

# Reduce dimensionality using SVD
svd_components = 10
svd = TruncatedSVD(n_components=svd_components, random_state=42)
reduced_matrix = svd.fit_transform(normalized_matrix)

print(f"Reduced matrix shape: {reduced_matrix.shape}")


In [ ]:
k_values = range(2, 50)  # Adjust the range of K as needed
silhouette_scores = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(reduced_matrix)
    labels = kmeans.labels_
    score = silhouette_score(reduced_matrix, labels)
    silhouette_scores.append(score)
    print(f"K={k}, Silhouette Score={score:.4f}")


In [ ]:
# Plot silhouette scores
plt.figure(figsize=(6, 4))
plt.plot(k_values, silhouette_scores, marker='o')
plt.title("Silhouette Scores for Different K")
plt.xlabel("Number of Clusters (K)")
plt.ylabel("Silhouette Score")
plt.grid()
plt.show()

# Find the optimal K
optimal_k = k_values[np.argmax(silhouette_scores)]
print(f"Optimal number of clusters (K): {optimal_k}")


In [ ]:
def silhouette_scores_cv_train_test(X, max_k=10, n_folds=10,metric='cosine'):
    """
    Performs KMeans clustering for k=2..max_k with 10-fold cross-validation.
    For each k, each fold is used as test once. We compute:
      - Silhouette on train set
      - Silhouette on test set
    Then average across the 10 folds and print.

    Finally, we plot the test silhouette scores by default.
    """
    k_values = range(2, max_k + 1)
    
    avg_train_scores = []
    avg_test_scores = []

    # For each k, run KFold cross-validation
    for k in k_values:
        fold_train_scores = []
        fold_test_scores  = []

        kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
        for train_idx, test_idx in kf.split(X):
            X_train, X_test = X[train_idx], X[test_idx]

            # Fit KMeans on train only
            kmeans = KMeans(n_clusters=k, random_state=42)
            kmeans.fit(X_train)

            # Predict labels
            labels_train = kmeans.predict(X_train)
            labels_test  = kmeans.predict(X_test)

            # Compute silhouette on train
            sil_train = silhouette_score(X_train, labels_train,metric='euclidean')
            fold_train_scores.append(sil_train)

            # Compute silhouette on test
            sil_test = silhouette_score(X_test, labels_test,metric=metric)
            fold_test_scores.append(sil_test)

        # Average across 10 folds
        # mean_train = np.mean(fold_train_scores)
        mean_test  = np.mean(fold_test_scores)
        
        # avg_train_scores.append(mean_train)
        avg_test_scores.append(mean_test)

        print(f"k={k}, Test Silhouette(avg)={mean_test:.4f}")

    # Plot the test scores by default
    # plt.figure(figsize=(7,5))
    # plt.plot(k_values, avg_test_scores, 'gx-')
    # plt.xlabel('k (Number of Clusters)')
    # plt.ylabel('Silhouette Score (Test Avg)')
    # plt.title('Silhouette Scores (Train/Test CV=10) - Test Plot')
    # plt.grid(True
    # plt.show()

    # (Optional) train vs test, :
    # plt.figure()
    # plt.plot(k_values, avg_train_scores, 'bo--', label='Train Score')
    # plt.plot(k_values, avg_test_scores,  'ro-', label='Test Score')
    # plt.xlabel('k')
    # plt.ylabel('Silhouette')
    # plt.title('Train vs Test Silhouette (CV=10)')
    # plt.legend()
    # plt.grid(True)
    # plt.show()

# Usage:
silhouette_scores_cv_train_test(X_norm, max_k=20, n_folds=10,metric='cosine')